#### Dataclasses

If you don't want to use _pydantic_'s `BaseModel` you can instead get the same data validation on standard `dataclasses` (introduced in Python 3.7).

In [1]:
import dataclasses
from datetime import datetime
from typing import List, Optional
from pydantic import Field
from pydantic.dataclasses import dataclass

In [2]:
@dataclass
class User:
    id: int
    name: str = "John Doe"
    signup_ts: datetime = None

In [3]:
user = User(id="42", signup_ts="2032-06-21T12:00")
print(f"{user = }")

user = User(id=42, name='John Doe', signup_ts=datetime.datetime(2032, 6, 21, 12, 0))


> ##### Note
>
> Keep in mind that `pydantic.dataclasses.dataclass` is a drop-in replacement for `dataclasses.dataclass` with validation, __not a replacement__ for `pydantic.BaseModel` (with a small difference in how `initialization hooks` work). There are cases where subclassing `pydantic.BaseModel` is the better choice.

You can use all the standard _pydantic_ field types, and the resulting dataclass will be identical to the one created by the standard library `dataclass` decorator.

The underlying model and its schema can be accessed through `__pydantic_model__`. Also, fields that require a `default_factory` can be specified by either a `pydantic.Field` or a `dataclasses.field`.

In [4]:
@dataclass
class ComplexUser:
    id: int
    name: str = "John Doe"
    friends: List[int] = dataclasses.field(default_factory=lambda: [0])
    age: Optional[int] = dataclasses.field(
        default=None,
        metadata=dict(title="The age of the user", description="do not lie!")
    )
    height: Optional[int] = Field(None, title="The height in cm", ge=50, le=300)

In [5]:
user = ComplexUser(id="42")
print(f"{user.__pydantic_model__.schema() = }")

user.__pydantic_model__.schema() = {'title': 'ComplexUser', 'type': 'object', 'properties': {'id': {'title': 'Id', 'type': 'integer'}, 'name': {'title': 'Name', 'default': 'John Doe', 'type': 'string'}, 'friends': {'title': 'Friends', 'type': 'array', 'items': {'type': 'integer'}}, 'age': {'title': 'The age of the user', 'description': 'do not lie!', 'type': 'integer'}, 'height': {'title': 'The height in cm', 'minimum': 50, 'maximum': 300, 'type': 'integer'}}, 'required': ['id']}


`pydantic.dataclasses.dataclass`'s arguments are the same as the standard decorator, except one extra keyword argument `config` which has the same meaning as `Config`.